In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_dropped = application_df.drop(['EIN', 'NAME'], axis=1)
application_df_dropped.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500
application_types_to_replace = application_df_dropped['APPLICATION_TYPE'].value_counts()[application_df_dropped['APPLICATION_TYPE'].value_counts() < cutoff].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df_dropped['APPLICATION_TYPE'] = application_df_dropped['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df_dropped['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [5]:
class_counts = application_df_dropped['CLASSIFICATION'].value_counts()

# Filter for classifications where the count is greater than 1
class_counts_greater_1 = class_counts[class_counts > 1]

# Display the result
class_counts_greater_1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_class = 1000
classifications_to_replace = application_df_dropped['CLASSIFICATION'].value_counts()[application_df_dropped['CLASSIFICATION'].value_counts() < cutoff_class].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_dropped['CLASSIFICATION'] = application_df_dropped['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df_dropped['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
X = pd.get_dummies(application_df_dropped)

In [8]:
# Split our preprocessed data into our features and target arrays
y = application_df_dropped['IS_SUCCESSFUL']  # Replace with the actual target column name

X = application_df_dropped.drop('IS_SUCCESSFUL', axis=1)  # Drop the target from features

X = pd.get_dummies(X)
#X
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#3 hidden nodes with increased neurons and differnt activation functions

nn = tf.keras.models.Sequential()

# First hidden layer and adding more neurons
nn.add(tf.keras.layers.Dense(units=256, input_dim=X_train_scaled.shape[1]))
nn.add(tf.keras.layers.LeakyReLU(alpha=0.01))  # Leaky ReLU instead of 'relu'

# Second hidden layer with tanh activation
nn.add(tf.keras.layers.Dense(units=128, activation='tanh'))

# Third hidden layer with relu activation
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 256)                 │          11,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 52,481 (205.00 KB)

 Trainable params: 52,481 (205.00 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Compile
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=150, batch_size=16)

Epoch 1/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 3s 856us/step - accuracy: 0.7142 - loss: 0.5803
Epoch 2/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 849us/step - accuracy: 0.7274 - loss: 0.5591
Epoch 3/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 855us/step - accuracy: 0.7291 - loss: 0.5561
Epoch 4/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step - accuracy: 0.7298 - loss: 0.5523
Epoch 5/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 837us/step - accuracy: 0.7274 - loss: 0.5541
Epoch 6/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.7309 - loss: 0.5508
Epoch 7/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - accuracy: 0.7356 - loss: 0.5463
Epoch 8/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.7332 - loss: 0.5427
Epoch 9/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 848us/step - accuracy: 0.7326 - loss: 0.5497
Epoch 10/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 857us/step - accuracy: 0.7283 - loss: 0.5488
Epoch 11/150
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.7394 - loss: 0.53

In [22]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7284 - loss: 0.5667
Loss: 0.5667053461074829, Accuracy: 0.728396475315094


In [23]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.h5')